In [1]:
import re
import nltk
import jpype
import string
import numpy as np
import pandas as pd
from sklearn import preprocessing
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,precision_score
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java

In [2]:
from gensim.models.fasttext import FastText

In [9]:
from sklearn.metrics import accuracy_score
from sklearn import model_selection, naive_bayes, svm

In [3]:
df = pd.read_csv("7allV03.csv")

In [4]:
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(df.category)

In [5]:
def tokenizasyon(text):
    return word_tokenize(text)
def lemmatizer(text):
    lemma_words = []
    ZEMBEREK_PATH = 'zemberek-full_old.jar'
    if jpype.isJVMStarted() is False:
        startJVM(getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))
    TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
    morphology = TurkishMorphology.createWithDefaults()
    for text in tokenizasyon(text):
        lemma_word = str(morphology.analyzeAndDisambiguate(str(text)).bestAnalysis()[0].getLemmas()[0])
        lemma_words.append(lemma_word)
    text = ' '.join(lemma_words)
    return text

def convert_lowercase(text):
    return text.lower()
def remove_punctuation(text):
    return ''.join(d for d in text if d not in string.punctuation)
def remove_stopwords(text):
    stopwords = []
    with open('stopwords.txt', 'r',encoding='utf-8') as f:
        for word in f:
            word = word.split('\n')
            stopwords.append(word[0])
    clean_text = ' '.join(s for  s in text.split() if s not in stopwords)
    return clean_text
def remove_numbers(text):
    text = re.sub(r'\d', '', text)
    return text
def remove_less_than_2(text):
    text = ' '.join([w for w in text.split() if len(w)>2])
    return text
def remove_extra_space(text):
    ornek_text_strip = re.sub(' +', ' ', text)
    return ornek_text_strip.strip()

In [6]:
df['text'] = df['text'].apply(lemmatizer)
df['text'] = df['text'].apply(convert_lowercase)
df['text'] = df['text'].apply(remove_punctuation)
df['text'] = df['text'].apply(remove_stopwords)
df['text'] = df['text'].apply(remove_extra_space)
df['text'] = df['text'].apply(remove_numbers)
df['text'] = df['text'].apply(remove_less_than_2)

In [7]:
X = df.text

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
corpus = []
for t in df.text:
    words = t.split()
    corpus.append([" ".join(words[i:i+1]) 
               for i in range(0, len(words), 1)])

In [11]:
ft_model = FastText(corpus, size=256, window=50, min_count=5, sg=1, iter=50)

In [12]:
ft_model.wv.most_similar("siyaset")

[('siyasi', 0.8612145781517029),
 ('şey', 0.8573281764984131),
 ('kim', 0.8504077792167664),
 ('hükümet', 0.8497319221496582),
 ('bütün', 0.845577597618103),
 ('parti', 0.842879056930542),
 ('alt', 0.8406243324279785),
 ('ana', 0.840239942073822),
 ('siz', 0.8385493755340576),
 ('olay', 0.8378793001174927)]

In [13]:
def vectorizer(model, X):
    return np.array([
        np.mean([model.wv[w] for w in document.split() if model.wv.__contains__(w)], axis=0)
        for document in X
    ])

In [14]:
feature_matrix = vectorizer(ft_model, X_train)

In [16]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(feature_matrix,y_train)

test_feature_matrix = vectorizer(ft_model, X_test)

predictions = SVM.predict(test_feature_matrix)
print("Accuracy Score -> ",accuracy_score(predictions, y_test)*100)

Accuracy Score ->  86.63265306122449
